In [1]:

import os, sys
import time
sys.path.insert(0, '..')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import lib
import torch, torch.nn as nn
import torch.nn.functional as F
from qhoptim.pyt import QHAdam

# read the data
data = pd.read_pickle('C:/Users/bcccu/Desktop/MQTTIDS/pkl/MQTTIDS_balanced.pkl')
# df = pd.read_csv('concatenated_data.csv')
print(f"Shape:",data['label'].value_counts())

device = 'cuda' if torch.cuda.is_available() else 'cpu'

Shape: label
Normal                             68890
encoded-payload-flood-Attack       40000
Connect-Disconnect-Flood-Attack    40000
publish-subscribe-flood-Attack     40000
Ping-Req-Flood-Attack              40000
slow-publish-Attack                40000
unvalid-publish-flood-Attack       40000
very-large-message-flood-Attack    40000
valid-publish-flood-Attack         40000
Name: count, dtype: int64


In [2]:
df = data.copy()
# Remove duplicate rows
df.drop_duplicates(inplace=True)

print(" Shape after removing duplicates:", df['label'].value_counts())
categorical_features = df.select_dtypes(include=['object']).columns.tolist()
print("Categorical features:", categorical_features)

# Drop only the columns that exist in the dataframe
columns_to_drop = ['flowid', 'TCPid', 'srcIP', 'dstIP', 'clientID', 'topic', 'startTime', 'endTime', 'username', 'password', 'fwdTobwdPacketRatio', 'bwdTofwdPacketRatio', 'willmsgLen', 'willtopicLen', 'protocolName', 'source_file']
df = df.drop(columns=[col for col in columns_to_drop if col in df.columns])
print("Columns dropped successfully! Shape:", df.shape)


 Shape after removing duplicates: label
Normal                             68882
encoded-payload-flood-Attack       40000
Connect-Disconnect-Flood-Attack    40000
publish-subscribe-flood-Attack     40000
Ping-Req-Flood-Attack              40000
slow-publish-Attack                40000
unvalid-publish-flood-Attack       40000
very-large-message-flood-Attack    40000
valid-publish-flood-Attack         40000
Name: count, dtype: int64
Categorical features: ['flowid', 'TCPid', 'srcIP', 'dstIP', 'clientID', 'topic', 'startTime', 'endTime', 'username', 'password', 'fwdTobwdPacketRatio', 'bwdTofwdPacketRatio', 'willmsgLen', 'willtopicLen', 'protocolName', 'label', 'source_file']
Columns dropped successfully! Shape: (388882, 389)


In [3]:
# Separate features and target
X = data.drop(columns=['label'])
y = data['label']

# Print dataset summary
print("Data Cleaning Done! Shape after removing duplicates:", X.shape)
print("Label distribution:\n", y.value_counts())

# Fill missing testues
for col in X.columns:
    if X[col].isnull().sum() > 0:
        if X[col].dtype == 'object':  # Categorical
            X[col] = X[col].fillna(X[col].mode()[0])  # Assign explicitly
            print("Column:", col, "is categorical")
        else:  # Numerical
            X[col] = X[col].fillna(X[col].median())  # Assign explicitly

print("Missing testues handled successfully!")
print("Shape after handling missing values:", X.shape)

Data Cleaning Done! Shape after removing duplicates: (388890, 404)
Label distribution:
 label
Normal                             68890
encoded-payload-flood-Attack       40000
Connect-Disconnect-Flood-Attack    40000
publish-subscribe-flood-Attack     40000
Ping-Req-Flood-Attack              40000
slow-publish-Attack                40000
unvalid-publish-flood-Attack       40000
very-large-message-flood-Attack    40000
valid-publish-flood-Attack         40000
Name: count, dtype: int64
Column: topic is categorical
Column: protocolName is categorical
Missing testues handled successfully!
Shape after handling missing values: (388890, 404)


In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report

# Separate features and target
X = data.drop(columns=['label'])  # Assuming 'label' is the target column
y = data['label']

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split data into 80% train, 20% test
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, stratify=y_encoded, random_state=42)

print("Data split completed!")
print(f"Train shape: {X_train.shape}, Test shape: {X_test.shape}")

# Ensure the LabelEncoder is fitted before accessing classes_
if hasattr(label_encoder, 'classes_'):
	class_names = label_encoder.classes_
else:
	label_encoder.fit(y)  # Fit the LabelEncoder if not already fitted
	class_names = label_encoder.classes_

print("Class names:", class_names)

from sklearn.preprocessing import LabelEncoder

# Combine train and test sets for encoding
combined = pd.concat([X_train, X_test], axis=0)

# Encode categorical features
label_encoder = LabelEncoder()
for col in combined.select_dtypes(include=['object']).columns:
	combined[col] = label_encoder.fit_transform(combined[col].astype(str))

# Split back into train and test sets
X_train = combined.iloc[:X_train.shape[0], :]
X_test = combined.iloc[X_train.shape[0]:, :]


for col in X_train.columns:
    if X_train[col].isnull().sum() > 0:
        if X_train[col].dtype == 'object':  # Categorical
            X_train[col] = X_train[col].fillna(X_train[col].mode()[0])  # Assign explicitly
            X_test[col] = X_test[col].fillna(X_test[col].mode()[0])  # Assign explicitly
        else:  # Numerical
            X_train[col] = X_train[col].fillna(X_train[col].median())  # Assign explicitly
            X_test[col] = X_test[col].fillna(X_test[col].median())  # Assign explicitly



# Convert to Float32 for PyTorch compatibility
X_train, X_test = X_train.astype(np.float32), X_test.astype(np.float32)

Data split completed!
Train shape: (311112, 404), Test shape: (77778, 404)
Class names: ['Connect-Disconnect-Flood-Attack' 'Normal' 'Ping-Req-Flood-Attack'
 'encoded-payload-flood-Attack' 'publish-subscribe-flood-Attack'
 'slow-publish-Attack' 'unvalid-publish-flood-Attack'
 'valid-publish-flood-Attack' 'very-large-message-flood-Attack']


C:\Users\bcccu\AppData\Local\Temp\ipykernel_2328\1835829437.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[col] = X_train[col].fillna(X_train[col].median())  # Assign explicitly
C:\Users\bcccu\AppData\Local\Temp\ipykernel_2328\1835829437.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test[col] = X_test[col].fillna(X_test[col].median())  # Assign explicitly
C:\Users\bcccu\AppData\Local\Temp\ipykernel_2328\1835829437.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy o

OPTUNA HYPERPARAMETER:


In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class GatedBlock(nn.Module):
    def __init__(self, in_features, out_features):
        super(GatedBlock, self).__init__()
        self.fc = nn.Linear(in_features, out_features)
        self.gate = nn.Linear(in_features, out_features)
        self._last_gate_output = None  # Make sure this line exists

    def forward(self, x):
        h = torch.tanh(self.fc(x))
        g = torch.sigmoid(self.gate(x))
        self._last_gate_output = g.detach()
        return h * g

        

class GANDALF(nn.Module):
    def __init__(self, input_dim, hidden_dims, output_dim, dropout=0.1):
        super(GANDALF, self).__init__()
        layers = []
        in_dim = input_dim

        for h_dim in hidden_dims:
            layers.append(GatedBlock(in_dim, h_dim))
            layers.append(nn.BatchNorm1d(h_dim))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout))
            in_dim = h_dim

        self.backbone = nn.Sequential(*layers)
        self.head = nn.Linear(in_dim, output_dim)

    def forward(self, x):
        x = self.backbone(x)
        return self.head(x)
    

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from torch.utils.data import TensorDataset, DataLoader

# Convert to tensors
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# Dataloaders
train_loader = DataLoader(TensorDataset(X_train_tensor, y_train_tensor), batch_size=1024, shuffle=True)
test_loader = DataLoader(TensorDataset(X_test_tensor, y_test_tensor), batch_size=1024, shuffle=False)

# Model, optimizer, loss
model = GANDALF(input_dim=X_train.shape[1], hidden_dims=[128, 64], output_dim=len(class_names), dropout=0.2)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)



In [ ]:
import optuna
import torch
import torch.nn as nn
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
import random

# --- assumes GatedBlock, GANDALF, X_train, y_train, X_test, y_test, class_names exist ---

# Tensors (if you don't already have them created above)
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor  = torch.tensor(X_test.values,  dtype=torch.float32)
y_test_tensor  = torch.tensor(y_test,  dtype=torch.long)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def set_seed(seed=42):
    random.seed(seed); np.random.seed(seed); torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

input_dim  = X_train.shape[1]
output_dim = len(class_names)

# ---------------------------
# Optuna objective (same format)
# ---------------------------
def objective(trial):
    set_seed(42)

    # Define hyperparameters to tune (TabNet-style dict)
    n_layers = trial.suggest_int("n_layers", 1, 4)
    hidden_dims = [trial.suggest_int(f"dim_{i+1}", 64, 512, step=64) for i in range(n_layers)]

    params = {
        "hidden_dims": hidden_dims,
        "dropout": trial.suggest_float("dropout", 0.0, 0.5),
        "optimizer_fn": trial.suggest_categorical("optimizer_fn", [torch.optim.Adam, torch.optim.AdamW]),
        "optimizer_params": {"lr": trial.suggest_float("lr", 1e-5, 5e-3, log=True),
                             "weight_decay": trial.suggest_float("weight_decay", 1e-8, 1e-2, log=True)},
        "batch_size": trial.suggest_categorical("batch_size", [256, 512, 1024, 2048]),
        "label_smoothing": trial.suggest_float("label_smoothing", 0.0, 0.1),
    }

    # Initialize GANDALF with suggested hyperparameters
    model = GANDALF(input_dim=input_dim,
                    hidden_dims=params["hidden_dims"],
                    output_dim=output_dim,
                    dropout=params["dropout"]).to(device)

    optimizer = params["optimizer_fn"](model.parameters(), **params["optimizer_params"])
    criterion = nn.CrossEntropyLoss(label_smoothing=params["label_smoothing"])

    # DataLoaders
    train_loader = DataLoader(TensorDataset(X_train_tensor, y_train_tensor),
                              batch_size=params["batch_size"], shuffle=True)

    # Train the model (mirrors your loop) with early stopping
    num_epochs = 100
    patience, wait = 10, 0
    best_acc = 0.0

    for _epoch in range(num_epochs):
        model.train()
        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)
            optimizer.zero_grad()
            preds = model(xb)
            loss = criterion(preds, yb)
            loss.backward()
            optimizer.step()

        # "Validation" on test split (same as your current setup)
        model.eval()
        with torch.no_grad():
            val_logits = model(X_test_tensor.to(device))
            val_acc = accuracy_score(y_test_tensor.cpu(), val_logits.argmax(dim=1).cpu())

        # early stopping signal
        if val_acc > best_acc:
            best_acc = val_acc
            wait = 0
        else:
            wait += 1
            if wait >= patience:
                break

    return best_acc

# ---------------------------
# Create & run the study
# ---------------------------
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=30)

print("Best Hyperparameters:", study.best_params)

# ---------------------------
# Train the final model with the best hyperparameters
# ---------------------------
bp = study.best_params
best_hidden = [bp[f"dim_{i+1}"] for i in range(bp["n_layers"])]

final_model = GANDALF(input_dim=input_dim, hidden_dims=best_hidden,
                      output_dim=output_dim, dropout=bp["dropout"]).to(device)

final_optimizer = bp["optimizer_fn"](final_model.parameters(),
                                     lr=bp["lr"], weight_decay=bp["weight_decay"])
final_criterion = nn.CrossEntropyLoss(label_smoothing=bp["label_smoothing"])

train_loader = DataLoader(TensorDataset(X_train_tensor, y_train_tensor),
                          batch_size=bp["batch_size"], shuffle=True)

num_epochs = 100
patience, wait = 10, 0
best_state, best_acc = None, 0.0

for epoch in range(num_epochs):
    final_model.train()
    total_loss = 0.0
    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)
        final_optimizer.zero_grad()
        preds = final_model(xb)
        loss = final_criterion(preds, yb)
        loss.backward()
        final_optimizer.step()
        total_loss += loss.item()

    final_model.eval()
    with torch.no_grad():
        val_logits = final_model(X_test_tensor.to(device))
        val_acc = accuracy_score(y_test_tensor.cpu(), val_logits.argmax(dim=1).cpu())

    print(f"[FINAL] Epoch {epoch+1}, Loss: {total_loss:.4f}, Val Acc: {val_acc:.4f}")

    if val_acc > best_acc:
        best_acc = val_acc
        wait = 0
        best_state = {k: v.cpu() for k, v in final_model.state_dict().items()}
    else:
        wait += 1
        if wait >= patience:
            print("Early stopping.")
            break

# Load & save the best weights
if best_state is not None:
    final_model.load_state_dict(best_state)
torch.save(final_model.state_dict(), "best_gandalf_optuna.pth")

# ---------------------------
# Evaluate the best model
# ---------------------------
final_model.eval()
with torch.no_grad():
    y_train_pred = final_model(X_train_tensor.to(device)).argmax(dim=1).cpu().numpy()
    y_test_pred  = final_model(X_test_tensor.to(device)).argmax(dim=1).cpu().numpy()

print("Training Accuracy:", accuracy_score(y_train, y_train_pred))
print("Test Accuracy:", accuracy_score(y_test, y_test_pred))
print("Test Classification Report:\n", classification_report(y_test, y_test_pred, target_names=class_names))
print("Confusion Matrix (Test):\n", confusion_matrix(y_test, y_test_pred))


[I 2025-09-19 10:17:53,648] A new study created in memory with name: no-name-bd95713d-d2d9-4854-9462-082df68359ea
c:\Users\bcccu\AppData\Local\Programs\Python\Python312\Lib\site-packages\optuna\distributions.py:515: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains <class 'torch.optim.adam.Adam'> which is of type type.
  warnings.warn(message)
c:\Users\bcccu\AppData\Local\Programs\Python\Python312\Lib\site-packages\optuna\distributions.py:515: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains <class 'torch.optim.adamw.AdamW'> which is of type type.
  warnings.warn(message)
[I 2025-09-19 10:41:30,092] Trial 0 finished with value: 0.9172259507829977 and parameters: {'n_layers': 3, 'dim_1': 128, 'dim_2': 384, 'dim_3': 384, 'dropout': 0.07617372981815962, 'optimizer_fn': <class 'torch.optim.adamw.AdamW'>, 'lr': 0

THE BEST ONE SO FAR

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class GatedBlock(nn.Module):
    def __init__(self, in_features, out_features):
        super(GatedBlock, self).__init__()
        self.fc = nn.Linear(in_features, out_features)
        self.gate = nn.Linear(in_features, out_features)
        self._last_gate_output = None  # Make sure this line exists

    def forward(self, x):
        h = torch.tanh(self.fc(x))
        g = torch.sigmoid(self.gate(x))
        self._last_gate_output = g.detach()
        return h * g

        

class GANDALF(nn.Module):
    def __init__(self, input_dim, hidden_dims, output_dim, dropout=0.1):
        super(GANDALF, self).__init__()
        layers = []
        in_dim = input_dim

        for h_dim in hidden_dims:
            layers.append(GatedBlock(in_dim, h_dim))
            layers.append(nn.BatchNorm1d(h_dim))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout))
            in_dim = h_dim

        self.backbone = nn.Sequential(*layers)
        self.head = nn.Linear(in_dim, output_dim)

    def forward(self, x):
        x = self.backbone(x)
        return self.head(x)


In [5]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from torch.utils.data import TensorDataset, DataLoader

# Convert to tensors
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# Dataloaders
train_loader = DataLoader(TensorDataset(X_train_tensor, y_train_tensor), batch_size=1024, shuffle=True)
test_loader = DataLoader(TensorDataset(X_test_tensor, y_test_tensor), batch_size=1024, shuffle=False)

# Model, optimizer, loss
model = GANDALF(input_dim=X_train.shape[1], hidden_dims=[128, 64], output_dim=len(class_names), dropout=0.2)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)


In [31]:
num_epochs = 100
best_acc = 0
patience, wait = 10, 0

for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        preds = model(xb)
        loss = criterion(preds, yb)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    # Validation
    model.eval()
    with torch.no_grad():
        val_preds = model(X_test_tensor.to(device))
        val_labels = y_test_tensor.to(device)
        val_acc = accuracy_score(val_labels.cpu(), val_preds.argmax(dim=1).cpu())

    print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}, Val Acc: {val_acc:.4f}")

    if val_acc > best_acc:
        best_acc = val_acc
        wait = 0
        torch.save(model.state_dict(), "First_best_gandalf_model.pth")
    else:
        wait += 1
        if wait >= patience:
            print("Early stopping.")
            break


Epoch 1, Loss: 38.5089, Val Acc: 0.7460
Epoch 2, Loss: 38.5806, Val Acc: 0.7463
Epoch 3, Loss: 38.2655, Val Acc: 0.7480
Epoch 4, Loss: 38.1592, Val Acc: 0.7467
Epoch 5, Loss: 38.1088, Val Acc: 0.7475
Epoch 6, Loss: 38.3226, Val Acc: 0.7484
Epoch 7, Loss: 37.9550, Val Acc: 0.7450
Epoch 8, Loss: 37.9684, Val Acc: 0.7477
Epoch 9, Loss: 38.1187, Val Acc: 0.7441
Epoch 10, Loss: 37.7968, Val Acc: 0.7474
Epoch 11, Loss: 38.3542, Val Acc: 0.7415
Epoch 12, Loss: 38.5347, Val Acc: 0.7401
Epoch 13, Loss: 38.2496, Val Acc: 0.7399
Epoch 14, Loss: 38.3976, Val Acc: 0.7412
Epoch 15, Loss: 38.2787, Val Acc: 0.7405
Epoch 16, Loss: 38.3077, Val Acc: 0.7413
Early stopping.


In [32]:
# Load best model
model.load_state_dict(torch.load("First_best_gandalf_model.pth"))
model.eval()

with torch.no_grad():
    y_train_pred = model(X_train_tensor.to(device)).argmax(dim=1).cpu().numpy()
    y_test_pred = model(X_test_tensor.to(device)).argmax(dim=1).cpu().numpy()

# Evaluation
print("Training Accuracy:", accuracy_score(y_train, y_train_pred))
print("Test Accuracy:", accuracy_score(y_test, y_test_pred))

print("Training Report:\n", classification_report(y_train, y_train_pred, target_names=class_names))
print("Test Report:\n", classification_report(y_test, y_test_pred, target_names=class_names))

print("Confusion Matrix (Test):\n", confusion_matrix(y_test, y_test_pred))


Training Accuracy: 0.7496118560338744
Test Accuracy: 0.7483770815692916
Training Report:
                 precision    recall  f1-score   support

   bf_dos/ddos       0.63      0.38      0.47      2400
    bruteforce       0.49      0.97      0.65      1183
delay_dos/ddos       0.78      0.21      0.34      2400
       malaria       0.98      0.99      0.99      2400
     malformed       0.98      0.96      0.97      2340
        normal       0.66      0.99      0.80      9598
       slowite       0.91      0.84      0.87       819
  sub_dos/ddos       0.99      0.71      0.83      2400
  syn_dos/ddos       0.64      0.16      0.26      2400
 will_dos/ddos       0.99      0.72      0.83      2400

      accuracy                           0.75     28340
     macro avg       0.81      0.69      0.70     28340
  weighted avg       0.78      0.75      0.72     28340

Test Report:
                 precision    recall  f1-score   support

   bf_dos/ddos       0.65      0.40      0.49       

C:\Users\bcccu\AppData\Local\Temp\ipykernel_32092\2284611206.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("First_best_gandalf_model.p

 Updated GANDALF Training Setup (Modified Hyperparameters

In [28]:
# Updated model with wider/deeper layers
model = GANDALF(
    input_dim=X_train.shape[1],
    hidden_dims=[256, 128, 64],  # deeper & wider
    output_dim=len(class_names),
    dropout=0.3  # stronger regularization
).to(device)

# Optimizer: Lower LR for stability + weight decay to reduce overfitting
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)

# Loss function (same)
criterion = nn.CrossEntropyLoss()

# DataLoader: Slightly smaller batch for better generalization
train_loader = DataLoader(TensorDataset(X_train_tensor, y_train_tensor), batch_size=512, shuffle=True)
test_loader = DataLoader(TensorDataset(X_test_tensor, y_test_tensor), batch_size=1024, shuffle=False)


INFO GAIN

In [43]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import mutual_info_classif
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd

# Step 1: Combine train and test for label encoding
combined = pd.concat([X_train, X_test], axis=0)
label_encoder = LabelEncoder()
for col in combined.select_dtypes(include=['object']).columns:
    combined[col] = label_encoder.fit_transform(combined[col].astype(str))

# Step 2: Split back to train/test
X_train_enc = combined.iloc[:X_train.shape[0], :]
X_test_enc = combined.iloc[X_train.shape[0]:, :]

# Fill missing values in encoded features (use median for numerical stability)
X_train_enc = X_train_enc.fillna(X_train_enc.median(numeric_only=True))

# Step 3: Compute Mutual Information (Information Gain)
mi_scores = mutual_info_classif(X_train_enc, y_train, discrete_features='auto', random_state=42)

# Step 4: Select Top-K Features
top_k_features = 100  # <-- change as needed
mi_sorted_idx = mi_scores.argsort()[::-1][:top_k_features]
selected_features = X_train_enc.columns[mi_sorted_idx]

# Step 5: Filter datasets
X_train_sel = X_train_enc[selected_features]
X_test_sel = X_test_enc[selected_features]

In [45]:
import torch
from torch.utils.data import TensorDataset, DataLoader

# Convert selected features to float32
X_train_tensor = torch.tensor(X_train_sel.values, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_sel.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)


batch_size = 1024

train_loader = DataLoader(TensorDataset(X_train_tensor, y_train_tensor), batch_size=batch_size, shuffle=True)
test_loader = DataLoader(TensorDataset(X_test_tensor, y_test_tensor), batch_size=batch_size, shuffle=False)


model = GANDALF(
    input_dim=X_train_sel.shape[1],  # Match top-K features
    hidden_dims=[128, 64],
    output_dim=len(class_names),
    dropout=0.1
).to(device)


optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# Optional: Class weighting
from sklearn.utils.class_weight import compute_class_weight
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
criterion = torch.nn.CrossEntropyLoss(weight=torch.tensor(class_weights, dtype=torch.float32).to(device))


from sklearn.metrics import accuracy_score

num_epochs = 100
best_acc = 0
patience, wait = 10, 0

for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        preds = model(xb)
        loss = criterion(preds, yb)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    # Validation
    model.eval()
    with torch.no_grad():
        val_preds = model(X_test_tensor.to(device)).argmax(dim=1).cpu().numpy()
        val_acc = accuracy_score(y_test, val_preds)

    print(f"Epoch {epoch+1:03d} | Loss: {total_loss:.4f} | Val Acc: {val_acc:.4f}")

    if val_acc > best_acc:
        best_acc = val_acc
        wait = 0
        torch.save(model.state_dict(), "best_gandalf_topk.pth")
    else:
        wait += 1
        if wait >= patience:
            print("🛑 Early stopping at epoch", epoch+1)
            break


Epoch 001 | Loss: 48.8873 | Val Acc: 0.5267
Epoch 002 | Loss: 35.8745 | Val Acc: 0.6047
Epoch 003 | Loss: 31.4497 | Val Acc: 0.6400
Epoch 004 | Loss: 28.7082 | Val Acc: 0.6664
Epoch 005 | Loss: 27.3662 | Val Acc: 0.6624
Epoch 006 | Loss: 25.9096 | Val Acc: 0.6510
Epoch 007 | Loss: 25.0478 | Val Acc: 0.6696
Epoch 008 | Loss: 24.9934 | Val Acc: 0.6612
Epoch 009 | Loss: 24.9830 | Val Acc: 0.6070
Epoch 010 | Loss: 25.7138 | Val Acc: 0.6540
Epoch 011 | Loss: 24.3425 | Val Acc: 0.6741
Epoch 012 | Loss: 24.0717 | Val Acc: 0.6703
Epoch 013 | Loss: 23.6246 | Val Acc: 0.7000
Epoch 014 | Loss: 23.1896 | Val Acc: 0.6874
Epoch 015 | Loss: 22.9041 | Val Acc: 0.6792
Epoch 016 | Loss: 22.9383 | Val Acc: 0.6509
Epoch 017 | Loss: 22.8957 | Val Acc: 0.6514
Epoch 018 | Loss: 22.4499 | Val Acc: 0.6734
Epoch 019 | Loss: 22.9359 | Val Acc: 0.6797
Epoch 020 | Loss: 22.4818 | Val Acc: 0.6675
Epoch 021 | Loss: 22.6516 | Val Acc: 0.6713
Epoch 022 | Loss: 22.3175 | Val Acc: 0.6729
Epoch 023 | Loss: 22.2418 | Val 

In [46]:
# Load best checkpoint
model.load_state_dict(torch.load("best_gandalf_topk.pth"))
model.eval()

with torch.no_grad():
    y_train_pred = model(X_train_tensor.to(device)).argmax(dim=1).cpu().numpy()
    y_test_pred = model(X_test_tensor.to(device)).argmax(dim=1).cpu().numpy()

from sklearn.metrics import classification_report, confusion_matrix

print("Training Accuracy:", accuracy_score(y_train, y_train_pred))
print("Test Accuracy:", accuracy_score(y_test, y_test_pred))

print("\nTraining Report:\n", classification_report(y_train, y_train_pred, target_names=class_names))
print("\nTest Report:\n", classification_report(y_test, y_test_pred, target_names=class_names))
print("\nConfusion Matrix (Test):\n", confusion_matrix(y_test, y_test_pred))


Training Accuracy: 0.7029640084685956
Test Accuracy: 0.6999717753316399

Training Report:
                 precision    recall  f1-score   support

   bf_dos/ddos       0.32      0.86      0.46      2400
    bruteforce       0.80      0.91      0.85      1183
delay_dos/ddos       0.73      0.13      0.22      2400
       malaria       0.96      0.94      0.95      2400
     malformed       0.80      0.95      0.87      2340
        normal       0.92      0.69      0.79      9598
       slowite       0.97      0.92      0.95       819
  sub_dos/ddos       0.90      0.66      0.76      2400
  syn_dos/ddos       0.41      0.55      0.47      2400
 will_dos/ddos       0.87      0.73      0.79      2400

      accuracy                           0.70     28340
     macro avg       0.77      0.73      0.71     28340
  weighted avg       0.79      0.70      0.71     28340


Test Report:
                 precision    recall  f1-score   support

   bf_dos/ddos       0.30      0.85      0.45     

C:\Users\bcccu\AppData\Local\Temp\ipykernel_32092\431686329.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("best_gandalf_topk.pth"))


In [39]:
import numpy as np
import torch
from sklearn.metrics import accuracy_score

num_epochs = 100
best_acc = 0
patience, wait = 10, 0

train_losses = []
val_accuracies = []

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0

    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)

        optimizer.zero_grad()
        preds = model(xb)
        loss = criterion(preds, yb)
        loss.backward()

        # Optional: gradient clipping (stable learning)
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=5)

        optimizer.step()
        epoch_loss += loss.item()

    avg_loss = epoch_loss / len(train_loader)
    train_losses.append(avg_loss)

    # Validation
    model.eval()
    with torch.no_grad():
        val_logits = model(X_test_tensor.to(device))
        val_preds = val_logits.argmax(dim=1).cpu().numpy()
        val_labels = y_test_tensor.cpu().numpy()

        val_acc = accuracy_score(val_labels, val_preds)
        val_accuracies.append(val_acc)

    print(f"Epoch {epoch+1:03d} | Loss: {avg_loss:.4f} | Val Acc: {val_acc:.4f}")

    # Save best model
    if val_acc > best_acc:
        best_acc = val_acc
        wait = 0
        torch.save(model.state_dict(), "First_best_gandalf_model.pth")
    else:
        wait += 1
        if wait >= patience:
            print("🛑 Early stopping triggered at epoch", epoch+1)
            break


Epoch 001 | Loss: 0.6786 | Val Acc: 0.7484
Epoch 002 | Loss: 0.6824 | Val Acc: 0.7480
Epoch 003 | Loss: 0.6827 | Val Acc: 0.7482
Epoch 004 | Loss: 0.6781 | Val Acc: 0.7482
Epoch 005 | Loss: 0.6791 | Val Acc: 0.7465
Epoch 006 | Loss: 0.6794 | Val Acc: 0.7482
Epoch 007 | Loss: 0.6786 | Val Acc: 0.7484
Epoch 008 | Loss: 0.6757 | Val Acc: 0.7485
Epoch 009 | Loss: 0.6809 | Val Acc: 0.7480
Epoch 010 | Loss: 0.6745 | Val Acc: 0.7480
Epoch 011 | Loss: 0.6802 | Val Acc: 0.7484
Epoch 012 | Loss: 0.6771 | Val Acc: 0.7481
Epoch 013 | Loss: 0.6783 | Val Acc: 0.7482
Epoch 014 | Loss: 0.6768 | Val Acc: 0.7482
Epoch 015 | Loss: 0.6786 | Val Acc: 0.7482
Epoch 016 | Loss: 0.6740 | Val Acc: 0.7484
Epoch 017 | Loss: 0.6736 | Val Acc: 0.7482
Epoch 018 | Loss: 0.6788 | Val Acc: 0.7481
🛑 Early stopping triggered at epoch 18


In [40]:
# Load best model
model.load_state_dict(torch.load("First_best_gandalf_model.pth"))
model.eval()

with torch.no_grad():
    y_train_pred = model(X_train_tensor.to(device)).argmax(dim=1).cpu().numpy()
    y_test_pred = model(X_test_tensor.to(device)).argmax(dim=1).cpu().numpy()

# Evaluation
print("Training Accuracy:", accuracy_score(y_train, y_train_pred))
print("Test Accuracy:", accuracy_score(y_test, y_test_pred))

print("Training Report:\n", classification_report(y_train, y_train_pred, target_names=class_names))
print("Test Report:\n", classification_report(y_test, y_test_pred, target_names=class_names))

print("Confusion Matrix (Test):\n", confusion_matrix(y_test, y_test_pred))


C:\Users\bcccu\AppData\Local\Temp\ipykernel_32092\2284611206.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("First_best_gandalf_model.p

Training Accuracy: 0.7496471418489767
Test Accuracy: 0.7485182049110923
Training Report:
                 precision    recall  f1-score   support

   bf_dos/ddos       0.63      0.38      0.47      2400
    bruteforce       0.49      0.97      0.65      1183
delay_dos/ddos       0.78      0.21      0.34      2400
       malaria       0.98      0.99      0.99      2400
     malformed       0.98      0.96      0.97      2340
        normal       0.66      0.99      0.80      9598
       slowite       0.91      0.84      0.87       819
  sub_dos/ddos       0.99      0.71      0.83      2400
  syn_dos/ddos       0.65      0.16      0.26      2400
 will_dos/ddos       0.99      0.72      0.83      2400

      accuracy                           0.75     28340
     macro avg       0.81      0.69      0.70     28340
  weighted avg       0.78      0.75      0.72     28340

Test Report:
                 precision    recall  f1-score   support

   bf_dos/ddos       0.65      0.40      0.49       

In [41]:
# Run a sample through the model
_ = model(X_test_tensor[:1].to(device))

# Access gate output from the first GatedBlock
first_gate_output = model.backbone[0]._last_gate_output.squeeze().cpu().numpy()

# Map to feature names
feature_gates = dict(zip(X_train.columns, first_gate_output))
sorted_gate_importance = dict(sorted(feature_gates.items(), key=lambda item: abs(item[1]), reverse=True))

# Print top-k
print("🔍 Top Gated Features (1 sample):")
for k, v in list(sorted_gate_importance.items())[:10]:
    print(f"{k}: {v:.4f}")


🔍 Top Gated Features (1 sample):
connackcount: 1.0000
pubackcount: 1.0000
pubrelcount: 1.0000
subscribecount: 1.0000
unsubscribecount: 1.0000
unsubackcount: 1.0000
pingrespcount: 1.0000
unsubscribepercentage: 1.0000
unsubackpercentage: 1.0000
disconnectpercentage: 1.0000
